In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from laspy.file import File

# Necessary to import pypwaves_updated.py from parent directory
import sys
sys.path.append('../')
import pypwaves_updated as pw

%matplotlib inline

## Comparing with LAS file
GPS timestamps kind of line up...the overall timeframe is identical to ~5 digits, but the offset is very different and the points don't line up exactly.  Also the point counts don't align, even accounting for pulses with multiple returns in the .las file.

In [5]:
# Load LAS points and header
las_df = pd.read_pickle("../../Data/pickles/las_points_164239.pkl")
las_header = File('../../Data/164239/10552_NYU_M2 - Scanner 1 - 190511_164239_1 - originalpoints.las', mode='r')

# Load pls anchor points and header
pls_anchor_df = pd.read_pickle("../../Data/pickles/anchor_points_164239.pkl")
pls_filename = "../../Data/164239_pls/10552_NYU_M2_Pulse_Waves_MTA_Resolved - Scanner 1 - 190511_164239_1 - originalpoints.pls"
pls_header = pw.openPLS(pls_filename)

# Adj GPS Time: Set both timestamps to zero for the first record
las_df['adj_gps_time'] = las_df['gps_time']-las_range[0]
pls_anchor_df['adj_gps_time'] = pls_anchor_df['gps_time']-anchor_range[0]

# Sort records by timestamp
las_df.sort_values(by=['adj_gps_time'],inplace=True)
pls_anchor_df.sort_values(by=['adj_gps_time'],inplace=True)

# Reset indices
las_df.reset_index(drop=True,inplace=True)
pls_anchor_df.reset_index(drop=True,inplace=True)

In [81]:
print("Number of points in .las file: {:,d}".format(las_df.shape[0]))
print("Number of pulses in .pls file: {:,d}".format(pls_anchor_df.shape[0]))

Number of points in .las file: 14,542,329
Number of pulses in .pls file: 10,710,919


### Comparing Timestamps
The count of points don't agree: LAS file has 25-50% more points.  Multiple returns per point account for some but not all the discrepancy.

However, the time ranges for the two files agree almost exactly.  Below, we try to line up these timestamps, but are generally unsuccessful.

In [76]:
# Compare time ranges
las_range = (las_df['gps_time'].min(),las_df['gps_time'].max())
anchor_range = (pls_anchor_df['gps_time'].min(),pls_anchor_df['gps_time'].max())

print("LAS range: {:2.3f} seconds".format(las_range[1]-las_range[0]))
print("PLS range: {:2.3f} seconds".format(anchor_range[1]-anchor_range[0]))

LAS range: 57.295 seconds
PLS range: 57.295 seconds


In [79]:
# The timestamps just don't line up very often.
print("PLS time\t LAS time")
for i in range(100,105):
    print("{:2.6f}\t {:2.6f}".format(pls_anchor_df['adj_gps_time'][i], las_df['adj_gps_time'][i]))

PLS time	 LAS time
0.001285	 0.001210
0.001288	 0.001218
0.001301	 0.001223
0.001303	 0.001226
0.001306	 0.001231


In [73]:
# Checking the GPS encoding for LAS file - Type 0 indicates GPS Week Time, otherwise GPS Adjusted Time.
# This seems to mean GPS Week Time
#print("LAS time type: ",las_header.header.get_gps_time_type())

print("\nLAS:")
print("GPS Time at start of run: {:2.3f} days".format(las_range[1]/ (60*60*24)))
# So the run was probably on a Sunday?
print("\nPLS:")
print("GPS Time at start of run: {:2.3f} days".format(anchor_range[0]/(60*60*24)))


LAS:
GPS Time at start of run: 0.697 days

PLS:
GPS Time at start of run: 6.696 days


### Timestamps - Analysis
Both systems could be on GPS Week time, in terms of order of magnitude of the values (both under 7 days), but they differ by 6 days.  I also haven't found any indication of when specifically the timestamp refers to: When the pulse was emitted, when it was received, "when" the point is, accounting for the time to the point itself, etc.

#### Idea: 
The difference could be that the pls file uses emit time and the point file adds the time to/from the point.

In [ ]:
# Noise points from anchor
noise_gps_times = anchor_df[anchor_df['z']<600]['gps_time_adj']
for time in noise_gps_times[:10000]:
    if time in las_df['gps_time_adj']:
        print("Yes!")

In [ ]:
noise_gps_times[1000:1010]

In [ ]:
las_df[las_df['gps_time_adj']>.37391][:10]

In [ ]:
las_df.iloc[1390:1408]

In [ ]:
plt.figure(figsize=(20,30))
plt.subplot(4, 1, 1)
plt.plot(las_df['x_scaled'][1390:1408])

plt.subplot(4, 1, 2)
plt.plot(las_df['y_scaled'][1390:1408])

plt.subplot(4, 1, 3)
plt.plot(las_df['z_scaled'][1390:1408])

plt.subplot(4, 1, 4)
plt.plot(las_df['intensity'][1390:1408])